In [9]:
#Libraries
import mysql.connector
import pandas as pd
from datetime import datetime
from tqdm import tqdm

#from secrets import dbpassword

In [10]:
#Initial Connection
cnx = mysql.connector.connect(user='root',
                              password='xxxx',
                              host='localhost',
                              database="SDPD"
                              )
cursor = cnx.cursor()

In [11]:
#function to download yearly data
def Get_Data(year):
    url = f"https://seshat.datasd.org/pd/pd_calls_for_service_{year}_datasd.csv"
    df = pd.read_csv(url)
    return df

In [15]:
url = f"http://seshat.datasd.org/pd/pd_cfs_calltypes_datasd.csv"
call_type = pd.read_csv(url).dropna(axis=1)
call_type

,call_type,description
0,1016,PRISONER IN CUSTODY
1,1016PT,PTU (PRISONER TRANSPORT)
2,1016QC,SHOPLIFTER/QUICK CITE
3,1021,PHONE YOUR STATION
4,1021H,PHONE YOUR HOME
...,...,...
284,TLO,TERRORISM LIAISON OFFICER
285,TP,TRAFFIC PURSUIT
286,TRO,"TRO VIOLATION, REPORT"
287,VARDA,VARDA ALARM


In [16]:
url = f"http://seshat.datasd.org/pd/pd_dispo_codes_datasd.csv"
disposition = pd.read_csv(url).dropna(axis=1)
disposition.loc[len(disposition.index)] = ['DEF', 'DEF'] 
disposition
disposition

,dispo_code,description
0,X,DISPATCHED ALARM CALL CANCELLED BEFORE UNIT ON...
1,A,ARREST MADE
2,AB,"ARREST MADE, BOARD AND SECURE"
3,AHR,"ARREST MADE, HIGH RISK (CRISIS RESPONSE TEAM)"
4,R,REPORT
5,RB,"REPORT TAKEN, BOARD AND SECURE"
6,RHR,"REPORT, HIGH RISK (CRISIS RESPONSE TEAM)"
7,S,STOLEN VEHICLE OR RECOVERED VEHICLE
8,O,OTHER
9,OHR,"OTHER, HIGH RISK (CRISIS RESPONSE TEAM)"


In [17]:
beat = pd.read_csv('beat-neighborhood.csv').dropna(axis=1)
beat_split = beat['beat,neighborhood'].str.split(pat=",",
                                                 expand=True)
beat_split = beat_split.rename(columns={0: "beat",
                                        1: "neighborhood"})
beat_split.loc[len(beat_split.index)] = [-1, 'No Beat'] 
beat_split

,beat,neighborhood
0,111,Clairemont Mesa East
1,112,Clairemont Mesa West
2,113,Bay Ho
3,114,North Clairemont
4,115,University City
...,...,...
121,934,Carmel Valley
122,935,North City
123,936,Torrey Highlands
124,937,Black Mountain Ranch


In [18]:
# Create dataframe
days = pd.DataFrame({
    'day_of_week':list(range(1,8)),
    'day':['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'],
})
days

,day_of_week,day
0,1,Sunday
1,2,Monday
2,3,Tuesday
3,4,Wednesday
4,5,Thursday
5,6,Friday
6,7,Saturday


In [21]:
# df to dict
call_type_dict = call_type.to_dict('records')
disposition_dict = disposition.to_dict('records')
beat_dict = beat_split.to_dict('records')
days_dict = days.to_dict('records')

In [22]:
def Find_Data(row,cursor,table, key):
    try:
        #print(f"SELECT * from {table} where {key} = '{row[key]}'")
        cursor.execute(f"SELECT * from {table} where {key} = '{row[key]}'")
        # Fetch a record
        result = cursor.fetchone()
    except:
        result = 0
    
    return result

In [25]:
#Call Type
def Load_Call_Type(row,cursor,table):
    # Create a new record
    sql = f"""INSERT INTO {table} (call_type,
                                    description
                                    ) VALUES (%s, %s)"""

    # Execute the query
    cursor.execute(sql, (row['call_type'],row['description']))
    cnx.commit()  

In [26]:
#Disposition
def Load_Dispo(row,cursor,table):
    # Create a new record
    sql = f"""INSERT INTO {table} (dispo_code,
                                    description
                                    ) VALUES (%s, %s)"""

    # Execute the query
    cursor.execute(sql, (row['dispo_code'],row['description']))
    cnx.commit()  

In [27]:
#beat
def Load_Beat(row,cursor,table):
    # Create a new record
    sql = f"""INSERT INTO {table} (beat,
                                    neighborhood
                                    ) VALUES (%s, %s)"""

    # Execute the query
    cursor.execute(sql, (row['beat'],row['neighborhood']))
    cnx.commit()  

In [28]:
#Load Data
def Load_Data(row,cursor,table,Col1,Col2):
    # Create a new record
    sql = f"""INSERT INTO {table} ({Col1},
                                    {Col2}
                                    ) VALUES (%s, %s)"""

    # Execute the query
    cursor.execute(sql, (row[Col1],row[Col2]))
    cnx.commit()  

In [29]:
for row in tqdm(call_type_dict):
    #Verify if Data is in table
    Data_Found = Find_Data(row,cursor,'Call_types','call_type')
    if Data_Found is None:
        #If no data found load 
        Load_Call_Type(row,cursor,'Call_types')
    else:
        pass

100%|███████████████████████████████████████| 289/289 [00:00<00:00, 1682.08it/s]


In [30]:
for row in tqdm(disposition_dict):
    #Verify if Data is in table
    Data_Found = Find_Data(row,cursor,'Dispositions','dispo_code')
    if Data_Found is None:
        #If no data found load 
        Load_Dispo(row,cursor,'Dispositions')
    else:
        pass

100%|██████████████████████████████████████████| 19/19 [00:00<00:00, 586.94it/s]


In [31]:
for row in tqdm(beat_dict):
    #Verify if Data is in table
    Data_Found = Find_Data(row,cursor,'Beats','beat')
    if Data_Found is None:
        #If no data found load 
        Load_Beat(row,cursor,'Beats')
    else:
        pass

100%|███████████████████████████████████████| 126/126 [00:00<00:00, 1353.28it/s]


In [ ]:
for row in tqdm(days_dict):
    Table = 'Days'
    Key = 'day_of_week'
    Col2 = 'day'
    #Verify if Data is in table
    Data_Found = Find_Data(row,cursor,Table,Key)
    if Data_Found is None:
        #If no data found load 
        Load_Data(row,cursor,Table,Key,Col2)
    else:
        pass

In [32]:
#Calls
def Load_Calls(row,cursor,table,year):
    # Create a new record
    sql = f"""INSERT INTO {table} ( incident_num,
                                    incident_date_time,
                                    day_of_week,
                                    call_type,
                                    dispo_code,
                                    beat,
                                    priority,
                                    incident_year
                                    ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""

    # Execute the query
    cursor.execute(sql, (row['incident_num'],
                         row['date_time'],
                         row['day_of_week'],
                         row['call_type'],
                         row['disposition'],
                         row['beat'],
                         row['priority'],
                         year,
                        )
                  )
    cnx.commit()  

In [33]:
Years = list(range(2022,datetime.now().year +1))
Years

[2022, 2023]

In [35]:
Years = [2021]
Years

[2021]

In [36]:
for year in Years:
    yearly_data = Get_Data(year)
    yearly_data_dict = yearly_data.to_dict('records')
    for row in tqdm(yearly_data_dict):
        Table = 'Service_calls'
        Key = 'incident_num'
        #Verify if Data is in table
        Data_Found = Find_Data(row,cursor,Table,Key)
        if Data_Found is None:
            #If no data found load
            try:
                Load_Calls(row,cursor,Table,year)
            except:
                print(row)
                

        else:
            print('error')
            print(row)

  0%|▏                                  | 2342/568947 [00:01<05:32, 1702.46it/s]

{'incident_num': 'E21010002172', 'date_time': '2021-01-02 12:18:55', 'day_of_week': 7, 'address_number_primary': 1600, 'address_dir_primary': nan, 'address_road_primary': 'PAC', 'address_sfx_primary': 'HWY', 'address_dir_intersecting': nan, 'address_road_intersecting': nan, 'address_sfx_intersecting': nan, 'call_type': 'DEMO', 'disposition': 'K', 'beat': 700, 'priority': 2}


  7%|██▌                               | 41855/568947 [00:24<05:12, 1687.33it/s]

{'incident_num': 'E21010044301', 'date_time': '2021-01-28 03:58:32', 'day_of_week': 5, 'address_number_primary': 5400, 'address_dir_primary': nan, 'address_road_primary': 'CASINO', 'address_sfx_primary': 'WY', 'address_dir_intersecting': nan, 'address_road_intersecting': nan, 'address_sfx_intersecting': nan, 'call_type': 'FU', 'disposition': 'K', 'beat': 300, 'priority': 2}


 12%|████▏                             | 69570/568947 [00:40<04:57, 1676.60it/s]

...

 99%|████████████████████████████████▋| 562806/568947 [05:28<00:03, 1709.34it/s]

{'incident_num': 'E21120038161', 'date_time': '2021-12-26 23:39:41', 'day_of_week': 1, 'address_number_primary': 1600, 'address_dir_primary': nan, 'address_road_primary': 'PAC', 'address_sfx_primary': 'HWY', 'address_dir_intersecting': nan, 'address_road_intersecting': nan, 'address_sfx_intersecting': nan, 'call_type': '1151', 'disposition': 'O', 'beat': 700, 'priority': 2}


 99%|████████████████████████████████▋| 563491/568947 [05:29<00:03, 1704.55it/s]

{'incident_num': 'E21120038900', 'date_time': '2021-12-27 13:05:40', 'day_of_week': 2, 'address_number_primary': 3700, 'address_dir_primary': nan, 'address_road_primary': 'ALABAMA', 'address_sfx_primary': 'ST', 'address_dir_intersecting': nan, 'address_road_intersecting': nan, 'address_sfx_intersecting': nan, 'call_type': '10851RR', 'disposition': 'SS-', 'beat': 813, 'priority': 3}


100%|█████████████████████████████████| 568947/568947 [05:32<00:00, 1711.90it/s]
